In [52]:
import pandas as pd
import re

def get_mid2rule():
    df = pd.read_json('../DATA/验证集/data.jsonl', lines=True)
    def get_rule(rule):
        rule = re.sub('该产品的|在各材料中的定义没有冲突|与', '', rule)
        return re.sub('的时间','时间',rule)

    df['rule'] = df['rule'].apply(get_rule)

    df = df.groupby('material_id').agg(tuple).reset_index()
    return dict(df[['material_id','rule']].values)

In [53]:
get_mid2rule()


{'m_00001a': ('基础产品销售信息',),
 'm_00001s': ('保障责任',),
 'm_00002s': ('责任免除', '基础产品销售信息'),
 'm_00003a': ('责任免除',),
 'm_00003s': ('责任免除',),
 'm_00004s': ('保障相关时间',),
 'm_00005a': ('投保条款',),
 'm_00005s': ('赔付 & 领取规则',),
 'm_00006s': ('保障相关时间',),
 'm_00007a': ('续保条款',),
 'm_00007s': ('投保条款',),
 'm_00008s': ('赔付 & 领取规则',),
 'm_00009s': ('基础产品销售信息',),
 'm_00010s': ('投保条款',),
 'm_00011s': ('附加条款',),
 'm_00012s': ('赔付 & 领取规则',),
 'm_00013s': ('责任免除',),
 'm_00014s': ('保障责任',),
 'm_00015s': ('保障相关时间',),
 'm_00016s': ('责任免除',),
 'm_00017s': ('保障责任',),
 'm_00018s': ('保障责任',),
 'm_00019s': ('投保条款',),
 'm_00061a': ('责任免除',),
 'm_00139a': ('基础产品销售信息',)}

In [56]:
'''
<thing>
### 分析

#### 文件1
文件1主要涉及的职业分类，列出了本保险不接受的职业范围，并明确了在这些职业范围内投保人故意或重大过失未履行如实告知义务时的处理方式：
- 不接受的职业包括：矿工、采石工、采砂工、爆破工、高压电工程设施人员、海洋船员、潜水员、火药炸药制造及处理人、特技演员、驯兽师、防暴警察、特种兵、战地记者、高空作业人员、脚手架上工作人员、室外安装维修人员、桥梁隧道施工人员、空中杂技人员、文员。
- 若投保人故意不履行如实告知义务，保险人不承担保险责任且不退还已交保险费。
- 若投保人因重大过失未履行如实告知义务，保险人不承担保险责任，将无息退还投保人所交保险费。

#### 文件2
文件2主要涉及的责任免除条款，列出了因以下情形导致被保险人发生保险事故时，保险公司不承担保险责任：
1. 投保人对被保险人的故意杀害、故意伤害；
2. 被保险人故意犯罪或抗拒依法采取的刑事强制措施；
3. 被保险人故意自伤、或自主险合同成立或者主险合同效力恢复之日起 2 年内自杀，但被保险人自杀时为无民事行为能力人的除外；
4. 被保险人服用、吸食或注射毒品；
5. 被保险人酒后驾驶、无合法有效驾驶证驾驶，或驾驶无合法有效行驶证的机动车；
6. 战争、军事冲突、暴乱或武装叛乱；
7. 核爆炸、核辐射或核污染；
8. 被保险人感染艾滋病病毒或患艾滋病；
9. 遗传性疾病，先天性畸形、变形或染色体异常。

### 比较
- **职业分类与责任免除**：文件1主要关注的是职业分类，而文件2主要关注的是特定行为或事件导致的保险事故。这两部分内容的侧重点不同，没有直接冲突。
- **处理方式**：文件1在投保人故意或重大过失未履行如实告知义务时的处理方式是不承担保险责任且不退还已交保险费或无息退还保险费。文件2在特定情形下不承担保险责任，但在某些情况下会退还现金价值。虽然处理方式不同，但适用的条件和背景不同，因此不存在直接冲突。

### 结论
两份文件在“责任免除”部分的描述和表达没有明显的相互冲突。文件1主要关注职业分类及其相关处理方式，而文件2主要关注特定行为或事件导致的保险事故及其处理方式。两者在不同的背景下有不同的规定，但不构成冲突。
</thing>
<result>一致</result>
'''

'\n<thing>\n### 分析\n\n#### 文件1\n文件1主要涉及的职业分类，列出了本保险不接受的职业范围，并明确了在这些职业范围内投保人故意或重大过失未履行如实告知义务时的处理方式：\n- 不接受的职业包括：矿工、采石工、采砂工、爆破工、高压电工程设施人员、海洋船员、潜水员、火药炸药制造及处理人、特技演员、驯兽师、防暴警察、特种兵、战地记者、高空作业人员、脚手架上工作人员、室外安装维修人员、桥梁隧道施工人员、空中杂技人员、文员。\n- 若投保人故意不履行如实告知义务，保险人不承担保险责任且不退还已交保险费。\n- 若投保人因重大过失未履行如实告知义务，保险人不承担保险责任，将无息退还投保人所交保险费。\n\n#### 文件2\n文件2主要涉及的责任免除条款，列出了因以下情形导致被保险人发生保险事故时，保险公司不承担保险责任：\n1. 投保人对被保险人的故意杀害、故意伤害；\n2. 被保险人故意犯罪或抗拒依法采取的刑事强制措施；\n3. 被保险人故意自伤、或自主险合同成立或者主险合同效力恢复之日起 2 年内自杀，但被保险人自杀时为无民事行为能力人的除外；\n4. 被保险人服用、吸食或注射毒品；\n5. 被保险人酒后驾驶、无合法有效驾驶证驾驶，或驾驶无合法有效行驶证的机动车；\n6. 战争、军事冲突、暴乱或武装叛乱；\n7. 核爆炸、核辐射或核污染；\n8. 被保险人感染艾滋病病毒或患艾滋病；\n9. 遗传性疾病，先天性畸形、变形或染色体异常。\n\n### 比较\n- **职业分类与责任免除**：文件1主要关注的是职业分类，而文件2主要关注的是特定行为或事件导致的保险事故。这两部分内容的侧重点不同，没有直接冲突。\n- **处理方式**：文件1在投保人故意或重大过失未履行如实告知义务时的处理方式是不承担保险责任且不退还已交保险费或无息退还保险费。文件2在特定情形下不承担保险责任，但在某些情况下会退还现金价值。虽然处理方式不同，但适用的条件和背景不同，因此不存在直接冲突。\n\n### 结论\n两份文件在“责任免除”部分的描述和表达没有明显的相互冲突。文件1主要关注职业分类及其相关处理方式，而文件2主要关注特定行为或事件导致的保险事故及其处理方式。两者在不同的背景下有不同的规定，但不构成冲突。\n</thing>\n<result>一致</result>\

In [57]:
import pandas as pd

df = pd.read_json(f"../outputs/submit.jsonl", lines=True)

In [63]:
type(df.iloc[0].material_id)

str

In [9]:
rule_clauses = {
    "基础产品销售信息": "该保险产品的基础配置信息，包括产品名、附加的条款信息、销售限制等",
    "投保条款": "投保过程中的缴费约定、投被保人条件限制等",
    "保障责任": "约定该产品的保险责任细节，如保障范围、保险金额、增值服务等",
    "保障相关时间": "约定该产品的各类时间信息，包括但不限于犹豫期、等待期、宽限期等",
    "赔付 & 领取规则": "约定该产品的保险责任的赔付、给付、领取及免赔细节，如赔付年龄/比例/次数等",
    "责任免除": "约定该产品不承担保险责任的情形,险人不承担给付保险金的责任",
    "续保条款": "约定续保相关信息，包括但不限于续保条件、保证续保等",
    "退保条款": "约定退保相关信息，包括但不限于退保条件、退保手续费等",
    "出险条款": "约定出险相关信息，包括但不限于出险地点、出险方式等",
    "附加条款": "约定该产品的附加条款，如特别约定等",
    "术语解释": "约定该产品的术语解释，如名词定义等"
}

{'m_00001s': '保障责任',
 'm_00002s': '基础产品销售信息',
 'm_00003s': '责任免除',
 'm_00004s': '保障相关时间',
 'm_00005s': '赔付 & 领取规则',
 'm_00006s': '保障相关时间',
 'm_00007s': '投保条款',
 'm_00008s': '赔付 & 领取规则',
 'm_00009s': '基础产品销售信息',
 'm_00010s': '投保条款',
 'm_00011s': '附加条款',
 'm_00012s': '赔付 & 领取规则',
 'm_00013s': '责任免除',
 'm_00014s': '保障责任',
 'm_00015s': '保障相关时间',
 'm_00016s': '责任免除',
 'm_00017s': '保障责任',
 'm_00018s': '保障责任',
 'm_00019s': '投保条款',
 'm_00001a': '基础产品销售信息',
 'm_00003a': '责任免除',
 'm_00005a': '投保条款',
 'm_00007a': '续保条款',
 'm_00139a': '基础产品销售信息',
 'm_00061a': '责任免除'}

In [19]:
rule_clauses.keys()

dict_keys(['基础产品销售信息', '投保条款', '保障责任', '保障相关时间', '赔付 & 领取规则', '责任免除', '续保条款', '退保条款', '出险条款', '附加条款', '术语解释'])